In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo

from sklearn.preprocessing import MinMaxScaler


In [21]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Function to center, scale, and return a series
def minmax_scale_group(group):
    median = group.median()
    centered_values = group - median
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(centered_values.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    medians[group.name] = median  # Store the median for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_minmax_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    median = medians[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values + median
    return pd.Series(original_values, index=group.index, name=group_name)

# Custom metric function
def multinacional_metric(y_true_scaled, y_pred_scaled):
    # Inverse transform and decenter the predicted values
    y_pred_original = y_pred_scaled.groupby(y_pred_scaled.index).apply(inverse_minmax_scale_group, group_keys=False)
    y_true_original = y_true_scaled.groupby(y_true_scaled.index).apply(inverse_minmax_scale_group, group_keys=False)
    # Calculate the metric using true values (already in the original scale)
    metric = abs(sum(y_true_original - y_pred_original)) / sum(y_true_original)
    
    return metric




In [22]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final = pd.read_parquet(DATOS_DIR+'/FE_02_dataset.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)


In [23]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2020-01-01']



#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]

In [26]:
df_final['tn_2'].fillna(0, inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\615317452.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['tn_2'].fillna(0, inplace=True)


In [27]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        #'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    
    ### Agrupar y escalar
    scalers = {}
    medians = {}
    df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
    df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse')
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_minmax_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


Fitting and predicting for product_id: 20524
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[54]	validation's rmse: 0.02289
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[26]	validation's rmse: 0.0367418
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[32]	validation's rmse: 0.049284
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[45]	validation's rmse: 0.0115015
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[51]	validation's rmse: 0.00615488
Overall rmse metric:  3.788254613515572e-05
             tn
20524  3.057227
Fitting and predicting for product_id: 20311
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[30]	validation's rmse: 0.0378684


C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	validation's rmse: 0.0315115
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[49]	validation's rmse: 0.037019
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[119]	validation's rmse: 0.0150008
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[25]	validation's rmse: 0.0248289
Overall rmse metric:  0.0002250232144551469
              tn
20311  15.196687
Fitting and predicting for product_id: 20654
Training until validation scores don't improve for 10 rounds


C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	validation's rmse: 0.00665604
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[63]	validation's rmse: 0.130011
Overall rmse metric:  2.6562374575471447e-05
            tn
20995  2.10331
Fitting and predicting for product_id: 20210
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	validation's rmse: 0.0428564


C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	validation's rmse: 0.182209
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[6]	validation's rmse: 0.0617832
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[13]	validation's rmse: 0.0376948
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	validation's rmse: 0.0775977
Overall rmse metric:  0.0014208994923316414
              tn
20210  59.157017
Fitting and predicting for product_id: 20127
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[14]	validation's rmse: 0.106408
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[32]	validation's rmse: 0.0541477
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[31]

C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_8540\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[24]	validation's rmse: 0.0257196
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[28]	validation's rmse: 0.0679735
Overall rmse metric:  0.0006614997944792931
               tn
20127  154.463717


In [28]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0012-prediccion-rmse_scaled_CORRECT-product_id.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)
print("Error prediccion 12-19: " , 
      abs(sum(df_true.groupby('product_id')['tn'].sum() - predictions_all['tn']))/sum(df_true.groupby('product_id')['tn'].sum()))

Overall custom metric:  0.0006614997944792931
Error prediccion 12-19:  0.004956122580158218
